In [ ]:
!pip install langchain

In [1]:
import os
import openai
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from dotenv import load_dotenv, find_dotenv

# load_dotenv(find_dotenv())
# openai.api_key = os.getenv("OPENAI_API_KEY")
# openai.api_base = os.getenv("OPENAI_API_BASE")
# openai.api_type = os.getenv("OPENAI_API_TYPE")
deployment_name='gpt-35-turbo_version_0301'

In [2]:
llm = OpenAI(deployment_id = deployment_name, max_tokens=150)
print(llm("Tell me a joke"))

WARNING! deployment_id is not default parameter.
                    deployment_id was transferred to model_kwargs.
                    Please confirm that deployment_id is what you intended.


.

Okay, here goes. Why did the computer go to the doctor?

I don't know.

Because it had a virus.

Oh, so cute. You're the best, Chloe.

I know, right?

See you later.

See you later, alligator. (computer beeps)

Hey, check out these hot wheels.

Hot wheels.

I don't know what they are, but they're pretty cool.

I told you, kids are so cute. (laughs)

You're so cute.

You're so cute.

You're so cute. (computer beeps)

Hello, are you my new best friend?

I've never used the word best before. I'm new to the whole friend thing.

Aw, you're cute.

Would


## PromptTemplates 
Prompt Templates makes creating a prompt for different usecases easier than using f-strings and interpolate the videos

In [ ]:
template = """
Interprete the text and evaluate the text.
sentiment: is the text in a positive, neutral or negative sentiment?
subject: What subject is the text about? Use exactly one word.

Format the output as JSON with the following keys:
sentiment
subject

text: {input}
"""

In [ ]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template=template)
chain = LLMChain(llm=llm, prompt=prompt_template)
print(chain.predict(input="I ordered Pizza Salami and it was awesome!"))

## Real World example with ResponseSchema, Templates, Chains and OutputParsers
There were two issues with the output: The output also contains text and the output is just a string which can not just the converted to a dictionary. Lets make it better with a more complex example

In [ ]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

sentiment_schema = ResponseSchema(
    name="sentiment",
    description="Is the text positive, neutral or negative? Only provide these words",
)
subject_schema = ResponseSchema(
    name="subject", description="What subject is the text about? Use exactly one word."
)
price_schema = ResponseSchema(
    name="price",
    description="How expensive was the product? Use None if no price was provided in the text",
)

response_schemas = [sentiment_schema, subject_schema, price_schema]
print(response_schemas)

In [ ]:
parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = parser.get_format_instructions()
print(format_instructions)

In [ ]:
template = """
Interprete the text and evaluate the text.
sentiment: is the text in a positive, neutral or negative sentiment?
subject: What subject is the text about? Use exactly one word.

Just return the JSON, do not add ANYTHING, NO INTERPRETATION!

text: {input}

{format_instructions}

"""

from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI

prompt = ChatPromptTemplate.from_template(template=template)

format_instructions = parser.get_format_instructions()

messages = prompt.format_messages(
    input="I ordered Pizza Salami for 9.99$ and it was awesome!",
    format_instructions=format_instructions,
)

chat = ChatOpenAI(deployment_id = deployment_name,temperature=0.0)
response = chat(messages)
print(response.content)


In [ ]:
output_dict = parser.parse(response.content)
output_dict